 <div style="text-align:center;">
   <span style="color:green; font-size:larger; font-weight:bold;">Exploratory Data Analysis (EDA)</span><br><br>
</div>


<span style="color:green; font-size:larger; font-weight:bold;">INTRODUCTION</span>

Dans cette partie, nous nous intéressons à l'analyse descriptive de la base de données "finale_merged" obtenue suite au nettoyage des bases de données initiales du fichier "Préparation_données_copy.ipynb"

In [2]:
# Importation des packages nécessaires
import declarations as d 

In [3]:
# Importation de la DF construite initialement dans le fichier "Préparation_données_copy.ipynb"
final_data= d.pd.read_csv('C:/Projet-Python/monmodule/bases/final_data.csv', index_col=0)

In [4]:
# Visualisation
final_data.head()

,COUNTRY,Unemployment_rate,GDP_rate,life_expentancy,pop_growth_rate
YEAR,,,,,
1994-01,AUS,9.814137,0.501645,77.878049,0.963092
1994-01,SVN,14.650000,0.748256,73.404878,-0.115694
1994-01,CAN,10.700000,0.680429,77.871707,1.095258
1994-01,AUT,6.609708,0.779464,76.419512,0.384870
1994-01,PHL,9.450000,0.329061,67.942000,2.343515


In [5]:
pivot_df = final_data.pivot_table(index='YEAR', columns='COUNTRY', values=['Unemployment_rate', 'GDP_rate','life_expentancy', 'pop_growth_rate'])
pivot_df.head()

GDP_rate                                                              \
COUNTRY       AUS       AUT       BEL       BGR       CAN       CHE       CZE   
YEAR                                                                            
1994-01  0.501645  0.779464  0.796550  0.559233  0.680429  0.907331  0.814153   
1994-04  0.504920  0.782479  0.800992  0.568045  0.689810  0.908917  0.822573   
1994-07       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1994-10       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1995-01  0.514092  0.727697  0.734561  0.021998  0.694571  0.910137  0.823188   

                                       ... pop_growth_rate            \
COUNTRY       DEU       ESP       EST  ...             PHL       POL   
YEAR                                   ...                             
1994-01  0.794548  0.774291  0.525902  ...        2.343515  0.211012   
1994-04  0.803800  0.786744  0.543380  ...        2.343515  0.211012   
1994-07       NaN       NaN       NaN  ...             NaN       NaN   
1994-10       NaN       NaN       NaN  ...             NaN       NaN   
1995-01  0.807890  0.635239  0.283429  ...        2.337837  0.135721   

                                                                               \
COUNTRY       PRT       ROU       RUS       SGP       SVK       SVN       SWE   
YEAR                                                                            
1994-01  0.269089 -0.145379 -0.034268  3.136587  0.394054 -0.115694  0.710705   
1994-04  0.269089 -0.145379 -0.034268  3.136587  0.394054 -0.115694  0.710705   
1994-07       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1994-10       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1995-01  0.346204 -0.202319 -0.021649  3.037813  0.292632  0.021561  0.524704   

                   
COUNTRY       USA  
YEAR               
1994-01  1.226296  
1994-04  1.226296  
1994-07       NaN  
1994-10       NaN  
1995-01  1.190787  

[5 rows x 118 columns]